# [Метрики качества классификации](https://www.coursera.org/learn/vvedenie-mashinnoe-obuchenie/programming/vfD6M/mietriki-kachiestva-klassifikatsii)

## Введение
В задачах классификации может быть много особенностей, влияющих на подсчет качества: различные цены ошибок, несбалансированность классов и т.д. Из-за этого существует большое количество метрик качества — каждая из них рассчитана на определенное сочетание свойств задачи и требований к ее решению.  

Меры качества классификации можно разбить на две большие группы: предназначенные для алгоритмов, выдающих номера классов, и для алгоритмов, выдающих оценки принадлежности к классам. К первой группе относятся доля правильных ответов, точность, полнота, F-мера. Ко второй — площади под ROC- или PR-кривой.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
data = pd.read_csv('./data/classification.csv', sep=",") 
true = data['true']
pred = data['pred']
data.head()

,true,pred
0,1,0
1,1,1
2,1,1
3,0,0
4,1,1


## 2. Заполните таблицу ошибок классификации:

In [2]:
TP = len(data[(data['pred'] == 1) & (data['true'] == 1)])
FP = len(data[(data['pred'] == 1) & (data['true'] == 0)])
FN = len(data[(data['pred'] == 0) & (data['true'] == 1)])
TN = len(data[(data['pred'] == 0) & (data['true'] == 0)])
print(f"TP: {TP} FP: {FP} FN: {FN} TN: {TN}")
pd.DataFrame({'Actual Positive': [TP, FN], 'Actual Negative': [FP, TN]}, index=['Predicted Positive', 'Predicted Negative'])

TP: 43 FP: 34 FN: 59 TN: 64


,Actual Positive,Actual Negative
Predicted Positive,43,34
Predicted Negative,59,64


## 3. Посчитайте основные метрики качества классификатора:

### Accuracy (доля верно угаданных) — sklearn.metrics.accuracy_score

In [3]:
from sklearn.metrics import accuracy_score
_ = accuracy_score(true, pred)
round(_, 2)

0.54

### Precision (точность) — sklearn.metrics.precision_score

In [4]:
from sklearn.metrics import precision_score
round(precision_score(true, pred), 2)

0.56

### Recall (полнота) — sklearn.metrics.recall_score

In [5]:
from sklearn.metrics import recall_score
round(recall_score(true, pred), 2)

0.42

### F-мера — sklearn.metrics.f1_score

In [6]:
from sklearn.metrics import f1_score
round(f1_score(true, pred), 2)

0.48

## 4. Имеется четыре обученных классификатора. В файле scores.csv записаны истинные классы и значения степени принадлежности положительному классу для каждого классификатора на некоторой выборке:
* для логистической регрессии — вероятность положительного класса (колонка score_logreg),
* для SVM — отступ от разделяющей поверхности (колонка score_svm),
* для метрического алгоритма — взвешенная сумма классов соседей (колонка score_knn),
* для решающего дерева — доля положительных объектов в листе (колонка score_tree).

In [7]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

data = pd.read_csv('./data/scores.csv', sep=",") 
true = data['true']
score_logreg = data['score_logreg']
score_svm = data['score_svm']
score_knn = data['score_knn']
score_tree = data['score_tree']
data.head()

,true,score_logreg,score_svm,score_knn,score_tree
0,0,0.683832,0.145976,0.787063,0.500000
1,1,0.801966,0.239511,1.000000,0.833333
2,0,0.382315,-0.245701,0.000000,0.000000
3,1,0.506797,-0.137058,0.000000,0.105263
4,1,0.488781,-0.154148,0.000000,0.105263


## 5. Посчитайте площадь под ROC-кривой для каждого классификатора. Какой классификатор имеет наибольшее значение метрики AUC-ROC (укажите название столбца)? Воспользуйтесь функцией sklearn.metrics.roc_auc_score.

In [8]:
roc_auc_score(true, score_logreg)

0.719187675070028

In [9]:
roc_auc_score(true, score_svm)

0.7086834733893557

In [10]:
roc_auc_score(true, score_knn)

0.6351540616246498

In [11]:
roc_auc_score(true, score_tree)

0.6919267707082833

## 6. Какой классификатор достигает наибольшей точности (Precision) при полноте (Recall) не менее 70% ? 

Чтобы получить ответ на этот вопрос, найдите все точки precision-recall-кривой с помощью функции sklearn.metrics.precision_recall_curve. Она возвращает три массива: precision, recall, thresholds. В них записаны точность и полнота при определенных порогах, указанных в массиве thresholds. Найдите максимальной значение точности среди тех записей, для которых полнота не меньше, чем 0.7.

In [17]:
from sklearn.metrics import precision_recall_curve, auc
import matplotlib.pyplot as plt    

In [32]:
precision, recall, thresholds = precision_recall_curve(true, score_logreg)
data = pd.DataFrame({'precision': precision , 'recall': recall})
print(data[data['recall'] >= 0.7]['precision'].idxmax())
print(data.loc[78])
# data.head()
data[70:80]
# max(data[data['recall'] >= 0.7]['precision'])

78
precision    0.630252
recall       0.765306
Name: 78, dtype: float64


,precision,recall
70,0.614173,0.795918
71,0.611111,0.785714
72,0.616000,0.785714
73,0.620968,0.785714
74,0.626016,0.785714
75,0.622951,0.775510
76,0.619835,0.765306
77,0.625000,0.765306
78,0.630252,0.765306
79,0.627119,0.755102


In [19]:
precision, recall, thresholds = precision_recall_curve(true, score_svm)
data = pd.DataFrame({'precision': precision , 'recall': recall})
print(data[data['recall'] >= 0.7]['precision'].idxmax())
print(data.loc[85])
data.head()

85
precision    0.622807
recall       0.724490
Name: 85, dtype: float64


,precision,recall
0,0.492462,1.000000
1,0.489899,0.989796
2,0.492386,0.989796
3,0.489796,0.979592
4,0.492308,0.979592


In [28]:
precision, recall, thresholds = precision_recall_curve(true, score_knn)
data = pd.DataFrame({'precision': precision , 'recall': recall})
print(data[data['recall'] >= 0.7]['precision'].idxmax())
print(data.loc[34])
# data.head()
data[20:40]

34
precision    0.606557
recall       0.755102
Name: 34, dtype: float64


,precision,recall
20,0.566176,0.785714
21,0.570370,0.785714
22,0.567164,0.775510
23,0.571429,0.775510
24,0.575758,0.775510
25,0.580153,0.775510
26,0.584615,0.775510
27,0.589147,0.775510
28,0.593750,0.775510
29,0.590551,0.765306


In [25]:
precision, recall, thresholds = precision_recall_curve(true, score_tree)
data = pd.DataFrame({'precision': precision , 'recall': recall})
print(data[data['recall'] >= 0.7]['precision'].idxmax())
print(data.loc[5])
data.head()

5
precision    0.651786
recall       0.744898
Name: 5, dtype: float64


,precision,recall
0,0.490000,1.000000
1,0.502732,0.938776
2,0.508475,0.918367
3,0.532895,0.826531
4,0.598485,0.806122
